In [8]:
import sys
sys.set_int_max_str_digits(0)

import json
from tqdm import tqdm
from datasets import load_dataset, Dataset

In [11]:
split = 'test'
ds = load_dataset("codeparrot/apps", "all", split=split)

In [12]:
instance_list = list()
for instance in tqdm(ds):
    try:
        problem_id = instance["problem_id"]
        question = instance["question"]
        solutions = instance["solutions"]
        input_output = instance["input_output"]
        difficulty = instance["difficulty"]
        starter_code = instance["starter_code"]

        if not input_output: continue
        if not solutions: continue
        input_output = json.loads(input_output)
        solutions = json.loads(instance["solutions"])

        test_cases = list()
        for input_, output_ in zip(input_output["inputs"], input_output["outputs"]):
            test_cases.append({
                'input': input_,
                'output': output_
            })

        instance = {
            'problem_id': int(problem_id),
            'question': str(question),
            'solutions': json.dumps(solutions),
            'test_cases': json.dumps(test_cases),
            'difficulty': str(difficulty),
            'starter_code': str(starter_code)
        }
        instance_list.append(instance)
    except Exception as e:
        print(e)

ds = Dataset.from_list(instance_list)
ds.push_to_hub("Elfsong/APPS", split=split)

100%|██████████| 5000/5000 [00:04<00:00, 1076.99it/s]


Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/476 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Elfsong/APPS/commit/008578fb8538e1c690c867e95e0d98d69c04c9c1', commit_message='Upload dataset', commit_description='', oid='008578fb8538e1c690c867e95e0d98d69c04c9c1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Elfsong/APPS', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Elfsong/APPS'), pr_revision=None, pr_num=None)

In [2]:
import requests

In [15]:
def post_code_submit(libs, code: str, timeout: int, profiling: bool) -> str:
    data = {
        'language': "python",
        'code': code,
        'libraries': libs,
        'timeout': timeout,
        'run_memory_profile': profiling
    }

    response = requests.post('https://monolith.cool/execute', json=data)
    task_id = response.json()['task_id']
    return task_id

def get_code_result(task_id: str) -> str:
    response = requests.get(f'https://monolith.cool/results/{task_id}')
    return response.json()

In [32]:
with open('apps_template.py', 'r') as f:
    code = f.read()

In [33]:
task_id = post_code_submit([], code, 60, False)

In [37]:
response = get_code_result(task_id)

In [39]:
print(response['output_dict']['stderr'].split('======================================================================')[0])

test_case_1 (__main__.TestSolution) ... ok
test_case_2 (__main__.TestSolution) ... ok
test_case_3 (__main__.TestSolution) ... FAIL




In [20]:
ds = load_dataset("Elfsong/APPS", split="test")

In [20]:
import json
import textwrap

In [21]:
solution = """
a = int(input())
b = int(input())
print(a + b)
""".strip()
solution = textwrap.indent(solution, "    ")
print(solution)

    a = int(input())
    b = int(input())
    print(a + b)


In [24]:
with open('apps_template', 'r') as f:
    TEMPLATE = f.read()

In [25]:
test_cases = [
    {"input": "1 2 3\n", "output": "6\n"},
    {"input": "10 20 30\n", "output": "60\n"}
]

test_case_list_str = json.dumps(test_cases, indent=4)

In [26]:
final_code = TEMPLATE.format(
    code_solution=solution,
    test_case_list=test_case_list_str
)

In [27]:
print(final_code)

import io
import sys
import unittest

def solution():
    a = int(input())
    b = int(input())
    print(a + b)

class TestSolution(unittest.TestCase):
    def run_io_fun(self, input_data):
        backup_stdin = sys.stdin
        backup_stdout = sys.stdout
        try:
            sys.stdin = io.StringIO(input_data)
            output_catcher = io.StringIO()
            sys.stdout = output_catcher

            solution()

            output_catcher.seek(0)
            return output_catcher.read()
        finally:
            sys.stdin = backup_stdin
            sys.stdout = backup_stdout

def make_test_function(input_data, expected):
    def test_function(self):
        actual = self.run_io_fun(input_data)
        self.assertEqual(expected, actual)
    return test_function

test_case_list = [
    {
        "input": "1 2 3\n",
        "output": "6\n"
    },
    {
        "input": "10 20 30\n",
        "output": "60\n"
    }
]

for i, case in enumerate(test_case_list, start=1):
    tes